# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [12]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [13]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lebu,-37.6167,-73.6500,12.14,75,3,6.66,CL,1701211098
1,1,saquarema,-22.9200,-42.5103,24.08,93,98,6.97,BR,1701211098
2,2,songyang,34.4553,113.0281,5.55,39,100,2.52,CN,1701211098
3,3,akureyri,65.6835,-18.0878,-0.04,100,75,0.00,IS,1701211099
4,4,altamont,42.2068,-121.7372,9.87,42,0,1.54,US,1701211099


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
%%capture --no-display
humidity = city_data_df["Humidity"].astype(int)
# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 27 degrees but higher than 21
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]

#Wind speed less than 4.5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 4.5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()


# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
154,154,kidal,18.4411,1.4078,23.96,12,0,3.78,ML,1701211131
254,254,banamba,13.5500,-7.4500,25.04,19,0,3.72,ML,1701211153
298,298,santa ana,33.7456,-117.8678,24.78,30,0,3.60,US,1701211163
310,310,wasco,35.5941,-119.3410,21.90,17,0,0.00,US,1701211166
559,559,katsuren-haebaru,26.3369,127.8719,21.43,60,0,3.09,JP,1701211221


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_dict={
    'City': city,
    'Country': country,
    'Lat': lat,
    'Lng': lng,
    'Humidity': humidity
}
hotel_df=pd.DataFrame(hotel_dict)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
154,kidal,ML,18.4411,1.4078,12,
254,banamba,ML,13.5500,-7.4500,19,
298,santa ana,US,33.7456,-117.8678,30,
310,wasco,US,35.5941,-119.3410,17,
559,katsuren-haebaru,JP,26.3369,127.8719,60,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lon = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address =  name_address.json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
kidal - nearest hotel: No hotel found
banamba - nearest hotel: No hotel found
santa ana - nearest hotel: Holiday Inn Express
wasco - nearest hotel: Best Western Plus
katsuren-haebaru - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
154,kidal,ML,18.4411,1.4078,12,No hotel found
254,banamba,ML,13.5500,-7.4500,19,No hotel found
298,santa ana,US,33.7456,-117.8678,30,Holiday Inn Express
310,wasco,US,35.5941,-119.3410,17,Best Western Plus
559,katsuren-haebaru,JP,26.3369,127.8719,60,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [18]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)